# COGS 108 - Data Checkpoint

# Names

- Brandon Vazquez
- Ernesto Escusa
- Chung En Pan
- Manuel Rodriguez Nunez
- Eric Estabaya

<a id='research_question'></a>
# Research Question

How has the COVID-19 infection rate influenced the video game community?

# Dataset(s)

- Dataset Name: covid-19-data (ouroworldindata.org) 16.1MB
- Link to the dataset: https://github.com/owid/covid-19-data/tree/master/public/data
- Number of observations: 62827 rows x 59 columns

This dataset is the global covid dataset, containing 192 countries infomation about the total of cases, new cases each day, total death, new death each day, total case per million, new case per million, new death/ total death per million, etc between 2020/1/1 to 2021/1/29.

- Dataset Name: Steam Games Dataset : Player count history, Price history and data about games 6.53GB
- Link to the dataset: https://data.mendeley.com/datasets/ycy3sy3vj2/1
- files Name: PlayercountHistoryPart1, PlayercountHistoryPart2, PriceHistory, Developers, Genres, Information, Packages, Supportedlanguages, Tags
- Number of observations:
    - PlayercountHistoryPart1: containing 1000 (game id) csvs, each of them 280224 rows x 2 column containing player count in each 5 minute interval
    - PlayercountHistoryPart2: containing 1000 (game id) csvs, each of them 23352 rows x 2 column containing player count in each 1 hour interval
    - PriceHistory: Containing 1513 (game id) csvs, each of them 493 rows x 4 column containing date, initial price, discounted price, and discouted rate
    - Developers: 2000 x 2 (game_id, developers)
    - Genres: 2000 x various (game_id, generes)
    - Information: 2000 x 5 (game_id, type, name, releasedate, freetoplay)
    - Packages: 1797 x 3 (game_id, package_id)
    - Supportedlanguages: 1993 x 1 (game_id, languages)
    - Tags: 2000 x 21 (game_id, tag1- 20)
    
    
 We planned to combine the steam data with its information, price history and the player count history, and support languages to get an understanding of each data. Thus, developers, genres, packages, tags csvs will probably be discarded. Steam data and covid data will be compared and investigated rather then combine.  



# Setup

- import os
- import pandas
- import blob
- import numpy

# Data Cleaning

For easier data wrangling and visualization later, both the playercounthistory1 and 2 'date' variable are transformed to datetime object, and the average are groupby by the day. Price history 'date' variable are transormed to datetime object and the price  and the rate average are groupby by the day. After transformation, the tables are saved as csv file and later merge with the information csv for later use. This process not only transforms the regular string object to datetime object, but also reduced the observation by a lot (original has more than 280000 observations per file, now to around 1000 observations per file for 2000 files).

In [ ]:
def wrangle_datetime(folder):

    empty_file= np.array([])
    filepath = '~/Github/covid_game_analysis/data/steaminfo/'+ folder
    
    for filename in os.listdir(filepath):
        filename_path = filepath + '/' + filename
        if filename.endswith('.csv'):
            if os.stat(filename_path).st_size !=0:
                with open(os.path.join(filepath, filename),'r+') as f:
                    print(filename)
                    temp= pd.read_csv(f,dtype={"Time": str, "PlayerCount": float})
                    temp['Time']= pd.to_datetime(temp['Time'], errors='coerce', format='%Y-%m-%d %H:%M')
                    temp = temp.dropna(axis=0)
                    result= temp.groupby([temp['Time'].dt.date]).mean()
                    result['Playercount']= result['Playercount'].apply(int)
                    result.to_csv(path_or_buf= filename_path)
                
        else:
            empty_file= np.append(empty_file,filename)
    print('empty_file', empty_file)
#wrangle_datetime('PlayerCountHistoryPart1')
#wrangle_datetime('PlayerCountHistoryPart2')

There are several files needed to be merged based on game id, so I extracted the game_id from information csv and went through the PlayerCountHistory1/2 to extract the time and playercount based on each game_id and then merge them to the corresponding game_id for 2000 game_id. Eventually what I obtained was a single table with game_id, names of the games, released date, time, playercount, etc.

In [ ]:
def mergeinfo():
    temp_info = pd.read_csv('data/steaminfo/Information.csv', encoding='unicode_escape')
    df = pd.DataFrame()
    empty_file= np.array([])
    for id in temp_info.get('appid'):
        
        file_path = 'data/steaminfo/PlayerCountHistory/' + str(id) + '.csv' 
        if os.stat(file_path).st_size !=0:
            with open(file_path, 'r') as f:
                f_csv = pd.read_csv(f,dtype={"Time": str, "Playercount": float})
                f_csv['appid'] = id
                temp = pd.DataFrame()
                temp = temp.append(temp_info[temp_info.get('appid') == id])
                temp = temp.merge(f_csv, how = 'inner', on = 'appid')
                df = df.append(temp)
                
        else:
            empty_file= np.append(empty_file,id)
    print('empty_file:',empty_file)
    df['Time']= pd.to_datetime(df['Time'], errors='coerce', format='%Y-%m-%d')
    return df
#df = mergeinfo()

Both of the function will notify me whether the files corresponding to the game_id are existed or has no content  in the file. 

# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/21 | 9:00 PM |    | Research on final project topic; find the hypothesis regarding the topics; divide tasks     |
| 1/28 | 9:30 PM | Research topic decided; divide tasks for project proposal; begin rough draft for proposal        | Finishing up rough draft project proposal, find the ideal dataset | 
| 2/5  | 9:00 PM | Search for potential datasets to be used in the project                                          | Discuss data that has been found, researching reliability and credibility |
| 2/12 | 9:00 PM | Continue to search for potential datasets, reducing and compacting data where necessary          | Discuss all data that has been found and what approach to take from this point, start data wrangling |
| 2/19 | ?????   | Continue wrangling data and begin EDA and data cleaning to format our needs                      | Discuss progress on data cleaning, check-in to ensure there are no issues |
| 2/26 | ?????   | Continue data cleaning for EDA checkpoint                                                        | Finalize data and prepare for EDA checkpoint submission, discuss data analysis |
| 3/5  | ?????   | Begin data analysis and interpretation of figures, charts, etc.                                  | Discuss progress on data analysis, check-in for any oversights and revisions to the dataset |
| 3/12 | ?????   | Continue data analysis and complete a rough draft of the analysis                                | Discuss revisions for the rough draft of the analysis, begin planning for video |
| 3/18 | ?????   | Work toward a final draft of the analysis, get a video script and recording                      | Finalize the data analysis and video portion of the project |
| 3/19 | Before 11:59 PM  | Fix typos and oversights in the analysis and video | Project submission |